In [1]:
import json
import re
import random
from datasets import load_dataset
import pandas as pd

import numpy as np
import transformers
from transformers import BertTokenizer, BertModel, get_linear_schedule_with_warmup
import torch
from sklearn.preprocessing import LabelEncoder
from tqdm import tqdm
import torch.nn as nn
from sklearn.metrics import accuracy_score
import pickle as pkl
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader
import string

import os
import pathlib
import re
import label_studio_sdk
import logging

from typing import List, Dict, Optional
from label_studio_ml.model import LabelStudioMLBase
from label_studio_ml.response import ModelResponse
import random
import string
from uuid import uuid4
import sys
from seqeval.metrics import f1_score, precision_score, recall_score, classification_report, accuracy_score


C:\ProgramData\anaconda3\Lib\site-packages\paramiko\pkey.py:82: CryptographyDeprecationWarning: TripleDES has been moved to cryptography.hazmat.decrepit.ciphers.algorithms.TripleDES and will be removed from this module in 48.0.0.
  "cipher": algorithms.TripleDES,
C:\ProgramData\anaconda3\Lib\site-packages\paramiko\transport.py:219: CryptographyDeprecationWarning: Blowfish has been moved to cryptography.hazmat.decrepit.ciphers.algorithms.Blowfish and will be removed from this module in 45.0.0.
  "class": algorithms.Blowfish,
C:\ProgramData\anaconda3\Lib\site-packages\paramiko\transport.py:243: CryptographyDeprecationWarning: TripleDES has been moved to cryptography.hazmat.decrepit.ciphers.algorithms.TripleDES and will be removed from this module in 48.0.0.
  "class": algorithms.TripleDES,


In [2]:

from transformers import pipeline, Pipeline
from itertools import groupby
from transformers import AutoModelForTokenClassification, TrainingArguments, Trainer, AutoTokenizer
from transformers import DataCollatorForTokenClassification
from datasets import Dataset, ClassLabel, Value, Sequence, Features
from functools import partial


In [3]:
logger = logging.getLogger(__name__)
_model: Optional[Pipeline] = None
MODEL_DIR = os.getenv('MODEL_DIR', './results')
BASELINE_MODEL_NAME = os.getenv('BASELINE_MODEL_NAME', 'Babelscape/wikineural-multilingual-ner')


FINETUNED_MODEL_NAME = os.getenv('FINETUNED_MODEL_NAME', 'rmdr_ner_model.bin')
CLS = [101]
SEP = [102]
VALUE_TOKEN = [0]
MAX_LEN = 512
TRAIN_BATCH_SIZE = 32
VAL_BATCH_SIZE = 8
EPOCHS = 15

In [4]:
# Метки для NER из LabelStudio
label_config = '''<View style="display:flex;align-items:start;gap:8px;flex-direction:row">
                   <Text name="text" value="$clearText" granularity="word"/>
                   <Labels name="label" toName="text" showInline="false">
                    <Label value="DIR" background="#4824f9"/>
                    <Label value="SLD" background="#00ff1e"/>
                    <Label value="WP" background="#ff0000"/>
                  	<Label value="LOC" background="#57fff4"/>
                    <Label value="UNIT" background="green"/>
                    <Label value="COUNT" background="#000000"/>
                    <Label value="FREE" background="#0008ff"/>
                    <Label value="LOST" background="#ff0000"/>
                    <Label value="CAPT" background="#ffbb00"/>
                  </Labels>
                </View> '''
project_id = 1

In [5]:
# Кусочек класса для работы с LabelStudio для получения задач и меток
class HuggingFaceNER(LabelStudioMLBase):
    """Custom ML Backend model
    """
    LABEL_STUDIO_HOST = os.getenv('LABEL_STUDIO_HOST', 'http://localhost:8080')
    LABEL_STUDIO_API_KEY = os.getenv('LABEL_STUDIO_API_KEY', 'c35a2f5689358d1e9d7522309643ba5b9cfca062')
    START_TRAINING_EACH_N_UPDATES = int(os.getenv('START_TRAINING_EACH_N_UPDATES', 10))
    LEARNING_RATE = float(os.getenv('LEARNING_RATE', 1e-3))
    NUM_TRAIN_EPOCHS = int(os.getenv('NUM_TRAIN_EPOCHS', 10))
    WEIGHT_DECAY = float(os.getenv('WEIGHT_DECAY', 0.01))

    def get_labels(self):
        li = self.label_interface
        from_name, _, _ = li.get_first_tag_occurence('Labels', 'Text')
        tag = li.get_tag(from_name)
        return tag.labels
    
    def _get_tasks(self, project_id):
        # download annotated tasks from Label Studio
        ls = label_studio_sdk.Client(self.LABEL_STUDIO_HOST, self.LABEL_STUDIO_API_KEY)
        project = ls.get_project(id=project_id)
        tasks = project.get_labeled_tasks()
        return tasks

In [6]:
# Класс для получение данных из LabelStudio (экзепляр)
ner =  HuggingFaceNER (project_id = project_id, label_config = label_config)

In [7]:
# Делаем словари меток с Id
labels = ['O'] + ner.get_labels()
id_to_label = {i: label for i, label in enumerate(labels)}
label_to_id = {label: i  for i, label in enumerate(labels)}

In [8]:
label_to_id

{'O': 0,
 'DIR': 1,
 'SLD': 2,
 'WP': 3,
 'LOC': 4,
 'UNIT': 5,
 'COUNT': 6,
 'FREE': 7,
 'LOST': 8,
 'CAPT': 9}

In [9]:
# Модель
class NERBertModel(nn.Module):
    
    def __init__(self, num_tag):
        super(NERBertModel, self).__init__()
        self.num_tag = num_tag
        self.bert = BertModel.from_pretrained(BASELINE_MODEL_NAME)
        self.bert_drop = nn.Dropout(0.3)
        self.out_tag = nn.Linear(768, self.num_tag)
        
    def forward(self, ids, mask, token_type_ids, target_tags):
        output, _ = self.bert(ids, attention_mask=mask, token_type_ids=token_type_ids, return_dict=False)
        bert_out = self.bert_drop(output) 
        tag = self.out_tag(bert_out)
    
        #Calculate the loss
        Critirion_Loss = nn.CrossEntropyLoss()
        active_loss = mask.view(-1) == 1
        active_logits = tag.view(-1, self.num_tag)
        active_labels = torch.where(active_loss, target_tags.view(-1), torch.tensor(Critirion_Loss.ignore_index).type_as(target_tags))
        loss = Critirion_Loss(active_logits, active_labels)
        return tag, loss

In [35]:
# функция загрузки готовой модели ( если уже есть)
def reload_model():
    global _model, _tokenizer, _device
    _model = None
    _tokenizer = None
    _device = None
    try:
        logger.info(f"Получаем девайс...")
        _device =  "cuda" if torch.cuda.is_available() else "cpu"
        logger.info(f"Девайс... {_device}")
        
        # Создаем токинайзер для нашей модели
        logger.info(f"Loading _tokenizer from {BASELINE_MODEL_NAME}")
        _tokenizer = AutoTokenizer.from_pretrained(BASELINE_MODEL_NAME)
        logger.info(f"_tokenizer from {BASELINE_MODEL_NAME} loaded")
        
        # загружаем модель
        model_path = str(pathlib.Path(MODEL_DIR) / FINETUNED_MODEL_NAME)
        print(f"Loading finetuned model from {model_path}")
        _model = NERBertModel(num_tag=len(label_to_id))
        _model.load_state_dict(torch.load(model_path, weights_only=True))
        _model.eval()
        print(f"...Loaded finetuned model from {model_path}")
    except Exception as e:
        # if finetuned model is not available, use the baseline model with the original labels
        logger.error(e)

In [36]:
reload_model()

Loading finetuned model from results\rmdr_ner_model.bin


Some weights of BertModel were not initialized from the model checkpoint at Babelscape/wikineural-multilingual-ner and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


...Loaded finetuned model from results\rmdr_ner_model.bin


In [12]:
# подготовка данных с метками в виде BIO. Не использую !!!!
# ds_raw = []
# no_label = 'O'
# label_to_id = {no_label: 0}
# for task in tasks:
#     for annotation in task['annotations']:
#         if not annotation.get('result'):
#             continue
#         # фильтруем результат - что бы была только разметака (value)
#         list_values = list(filter(lambda r: r['type'] == 'labels', annotation['result']))
#         spans = [{'label': r['value']['labels'][0], 'start': r['value']['start'], 'end': r['value']['end']} for r in list_values]
#         spans = sorted(spans, key=lambda x: x['start'])
#         text = ner.preload_task_data(task, task['data'][value])
#         #text = text[:MAX_LEN]
#         # insert tokenizer.pad_token to the unlabeled chunks of the text in-between the labeled spans, as well as to the beginning and end of the text
#         last_end = 0
#         all_spans = []
#         for span in spans:
#             if last_end < span['start']:
#                 all_spans.append({'label': no_label, 'start': last_end, 'end': span['start']})
#             all_spans.append(span)
#             last_end = span['end']
#         if last_end < len(text):
#             all_spans.append({'label': no_label, 'start': last_end, 'end': len(text)})
    
#         # now tokenize chunks separately and add them to the dataset
#         item = {'id': task['id'], 'tokens': [], 'tokens_ids': [], 'ner_tags': [], value: task['data'][value]}
#         for span in all_spans:
#             tokens = tokenizer.tokenize(text[span['start']:span['end']])
#             tokens_ids = tokenizer.encode(text[span['start']:span['end']], add_special_tokens=False)
#             item['tokens'].extend(tokens)
#             item['tokens_ids'].extend(tokens_ids)
#             if span['label'] == no_label:
#                 item['ner_tags'].extend([label_to_id[no_label]] * len(tokens))
#                 if len(item['tokens']) != len(item['ner_tags']): 
#                     print (len(item['tokens']), item['tokens'])
#                     print (len(item['ner_tags']), item['ner_tags'])
#                     print(text[span['start']:span['end']])
#                     print(tokens)
#             else:
#                 label = 'B-' + span['label']
#                 if label not in label_to_id:
#                     label_to_id[label] = len(label_to_id)
#                 item['ner_tags'].append(label_to_id[label])
#                 if len(tokens) > 1:
#                     label = 'I-' + span['label']
#                     #print(span)
#                     #print (len(tokens), tokens)
#                     if label not in label_to_id:
#                         label_to_id[label] = len(label_to_id)
#                     item['ner_tags'].extend([label_to_id[label] for _ in range(1, len(tokens))])
                   
#         ds_raw.append(item)


In [13]:
# Получаем задачи из LabelStudio
tasks = ner._get_tasks(project_id)

In [14]:
# получение данных для модели (потом еще будет DataSet на базе этого списка)
li = ner.label_interface
from_name, to_name, value = li.get_first_tag_occurence('Labels', 'Text')
ds_raw = []
no_label = 'O'
for task in tasks:
    for annotation in task['annotations']:
        if not annotation.get('result'):
            continue
        # фильтруем результат - что бы была только разметака (value)
        list_values = list(filter(lambda r: r['type'] == 'labels', annotation['result']))
        spans = [{'label': r['value']['labels'][0], 'start': r['value']['start'], 'end': r['value']['end']} for r in list_values]
        spans = sorted(spans, key=lambda x: x['start'])
        text = ner.preload_task_data(task, task['data'][value])
        #text = text[:MAX_LEN]
        # insert tokenizer.pad_token to the unlabeled chunks of the text in-between the labeled spans, as well as to the beginning and end of the text
        last_end = 0
        all_spans = []
        for span in spans:
            if last_end < span['start']:
                all_spans.append({'label': no_label, 'start': last_end, 'end': span['start']})
            all_spans.append(span)
            last_end = span['end']
        if last_end < len(text):
            all_spans.append({'label': no_label, 'start': last_end, 'end': len(text)})
    
        # now tokenize chunks separately and add them to the dataset
        item = {'id': task['id'], 'tokens': [], 'tokens_ids': [], 'ner_tags': [], value: task['data'][value]}
        for span in all_spans:
            tokens = _tokenizer.tokenize(text[span['start']:span['end']])
            tokens_ids = _tokenizer.encode(text[span['start']:span['end']], add_special_tokens=False)
            item['tokens'].extend(tokens)
            item['tokens_ids'].extend(tokens_ids)
            if span['label'] == no_label:
                item['ner_tags'].extend([label_to_id[no_label]] * len(tokens))
                if len(item['tokens']) != len(item['ner_tags']): 
                    print (len(item['tokens']), item['tokens'])
                    print (len(item['ner_tags']), item['ner_tags'])
                    print(text[span['start']:span['end']])
                    print(tokens)
            else:
                label = span['label']
                item['ner_tags'].append(label_to_id[label])
                if len(tokens) > 1:
                    label = span['label']
                    item['ner_tags'].extend([label_to_id[label] for _ in range(1, len(tokens))])
                   
        ds_raw.append(item)


Token indices sequence length is longer than the specified maximum sequence length for this model (679 > 512). Running this sequence through the model will result in indexing errors


In [15]:
# Разбиваем тексты (списки токенов) которые больше 512 на куски меньше, со сдвигом назад на SHIFT_SIZE. 
SHIFT_SIZE = 200
def make_ds_raw_ext (_ds_raw):
    ds_raw_ext = []
    for item in _ds_raw:
        tokens_len = len (item['tokens'])
        chank_index = 0
        for token_index in range(0, tokens_len, MAX_LEN-SHIFT_SIZE):
            token_index_end = token_index+MAX_LEN
            #print(token_index, token_index_end,  tokens_len, chank_index)
            item_ext = {'id': item['id'], 
                        'tokens': item['tokens'][token_index:token_index_end], 
                        'tokens_ids': item['tokens_ids'][token_index:token_index_end], 
                        'ner_tags': item['ner_tags'][token_index:token_index_end], 
                        value: item[value]}
            ds_raw_ext.append(item_ext)
            chank_index = chank_index + 1
    return ds_raw_ext

In [16]:
ds_raw_ext = make_ds_raw_ext (ds_raw)

In [17]:
print(len(ds_raw) , len(ds_raw_ext) )

164 478


In [18]:
print(len(ds_raw[5]['tokens']),len(ds_raw[5]['ner_tags']))
print(len(ds_raw_ext[5]['tokens']),len(ds_raw_ext[5]['ner_tags']))

1040 1040
162 162


In [19]:
#ds_raw_ext[6]

In [18]:
# Датасет на базе ds_raw. Будет передаваться на обучение
class Dataset:
  
  def __init__(self, ds_raw):
    #ds_raw item = {'id': task['id'], 'tokens': [], 'tokens_ids': [], 'ner_tags': [], value: task['data'][value]}
    self.ds_raw = ds_raw
  def __len__(self):
    return len(self.ds_raw)

  def __getitem__(self, index):
    ds_raw_item = self.ds_raw[index]
    #Tokenise
    ids = ds_raw_item['tokens_ids']
    target_tag = ds_raw_item['ner_tags']
    #To Add Special Tokens, subtract 2 from MAX_LEN
    ids = ids[:MAX_LEN - 2]
    target_tag = target_tag[:MAX_LEN - 2]
    #Add Sepcial Tokens
    ids = CLS + ids + SEP
    target_tags = VALUE_TOKEN + target_tag + VALUE_TOKEN
    mask = [1] * len(ids)
    token_type_ids = [0] * len(ids)
    #Add Padding if the input_len is small
    padding_len = MAX_LEN - len(ids)
    ids = ids + ([0] * padding_len)
    target_tags = target_tags + ([0] * padding_len)
    mask = mask + ([0] * padding_len)
    token_type_ids = token_type_ids + ([0] * padding_len)
    return {
        "ids" : torch.tensor(ids, dtype=torch.long),
        "mask" : torch.tensor(mask, dtype=torch.long),
        "token_type_ids" : torch.tensor(token_type_ids, dtype=torch.long),
        "target_tags" : torch.tensor(target_tags, dtype=torch.long)
      }

In [19]:
def compute_metrics(eval_prediction):
    predictions, labels = eval_prediction
    predictions = predictions.argmax(2)

    predictions = predictions.cpu().numpy()
    labels = labels.cpu().numpy()

    # Remove ignored index (special tokens)
    true_predictions = [
        [id_to_label[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [id_to_label[l]  for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]


    return {
        "precision": precision_score(true_labels, true_predictions),
        "recall": recall_score(true_labels, true_predictions),
        "f1": f1_score(true_labels, true_predictions),
        "classification_report": classification_report(true_labels, true_predictions),
        "accuracy_score" : accuracy_score(true_labels, true_predictions),
    }


In [20]:
# функця обучения
def train_fn(train_data_loader, model, optimizer, device, scheduler):
    #Train the Model
    model.train()
    loss_ = 0
    metrics_ = {
        "precision":0,
        "recall":0,
        "accuracy_score":0,
        # "f1": 0,
        # "classification_report": 0,
    }
    for data in tqdm(train_data_loader, total = len(train_data_loader)):
        for i, j in data.items():
            data[i] = j.to(device)
        target_tags = data["target_tags"]
        #Backward Propagation
        optimizer.zero_grad()
        tag, loss = model(**data)
        #print(tag)
        # tag2 = tag.argmax(2)
        # print(tag2)
        loss.backward()
        optimizer.step()
        scheduler.step()
        loss_ += loss.item()
        metrics = compute_metrics((tag, target_tags))
        metrics_["precision"] = metrics_["precision"] + float(metrics["precision"])
        metrics_["recall"] = metrics_["recall"] + float(metrics["recall"])
        metrics_["accuracy_score"] = metrics_["accuracy_score"] + float(metrics["accuracy_score"])
        #print (metrics)
    metrics_["precision"] = metrics_["precision"]  / len(train_data_loader)
    metrics_["recall"] = metrics_["recall"]  / len(train_data_loader)
    metrics_["accuracy_score"] = metrics_["accuracy_score"]  / len(train_data_loader)
    return model, loss_ / len(train_data_loader), metrics_
# функйия валидации
def val_fn(val_data_loader, model, optimizer, device, scheduler):
    model.eval()
    loss_ = 0
    metrics_ = {
        "precision":0,
        "recall":0,
        "accuracy_score":0,
        # "f1": 0,
        # "classification_report": 0,
    }
    for data in tqdm(val_data_loader, total = len(val_data_loader)):
        for i, j in data.items():
            data[i] = j.to(device)
        target_tags = data["target_tags"]
        tag, loss = model(**data)
        metrics = compute_metrics((tag, target_tags))
        metrics_["precision"] = metrics_["precision"] + float(metrics["precision"])
        metrics_["recall"] = metrics_["recall"] + float(metrics["recall"])
        metrics_["accuracy_score"] = metrics_["accuracy_score"] + float(metrics["accuracy_score"])
        loss_ += loss.item()
    metrics_["precision"] = metrics_["precision"]  / len(val_data_loader)
    metrics_["recall"] = metrics_["recall"]  / len(val_data_loader)
    metrics_["accuracy_score"] = metrics_["accuracy_score"]  / len(val_data_loader)
    return loss_ / len(val_data_loader), metrics_

In [21]:
#Train Test Split
train_ds_raw, val_ds_raw= train_test_split(ds_raw_ext, test_size=0.1, random_state=10)

In [22]:
#Create DataLoaders
train_dataset = Dataset(ds_raw = train_ds_raw)
val_dataset = Dataset(ds_raw = val_ds_raw)
train_data_loader = DataLoader(train_dataset, batch_size=TRAIN_BATCH_SIZE)
val_data_loader = DataLoader(val_dataset, batch_size=VAL_BATCH_SIZE)

In [25]:
# проверка лоадера
# for data_ in train_data_loader:
#     print(data_)
#     break

In [26]:
# Экзепляр модели для обучения
model = NERBertModel(num_tag=len(label_to_id))
model.to(_device)

Some weights of BertModel were not initialized from the model checkpoint at Babelscape/wikineural-multilingual-ner and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


NERBertModel(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(119547, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise

In [27]:
#Function for getparameters
def get_hyperparameters(model, ff):

    # ff: full_finetuning
    if ff:
        param_optimizer = list(model.named_parameters())
        no_decay = ["bias", "gamma", "beta"]
        optimizer_grouped_parameters = [
            {
                "params": [
                    p for n, p in param_optimizer if not any(nd in n for nd in no_decay)
                ],
                "weight_decay_rate": 0.01,
            },
            {
                "params": [
                    p for n, p in param_optimizer if any(nd in n for nd in no_decay)
                ],
                "weight_decay_rate": 0.0,
            },
        ]
    else:
        # Делаем все параметры не обучаемыми
        for param in model.bert.parameters():
            param.requires_grad = False
        
        # Последнии слои модели делаем обучаемыми (сколько влазит в GPU)
        param_optimizer = list(model.out_tag.parameters())
        for param in param_optimizer:
            param.requires_grad = True
        optimizer_grouped_parameters = [{"params": [p for p in param_optimizer]}]

        for param in list(model.bert.pooler.parameters()):
            param.requires_grad = True
            optimizer_grouped_parameters[0]["params"].append(param)

        for param in list(model.bert.encoder.layer[3].parameters()):
            param.requires_grad = True
            optimizer_grouped_parameters[0]["params"].append(param)

        for param in list(model.bert.encoder.layer[4].parameters()):
            param.requires_grad = True
            optimizer_grouped_parameters[0]["params"].append(param)

        for param in list(model.bert.encoder.layer[5].parameters()):
            param.requires_grad = True
            optimizer_grouped_parameters[0]["params"].append(param)

        for param in list(model.bert.encoder.layer[6].parameters()):
            param.requires_grad = True
            optimizer_grouped_parameters[0]["params"].append(param)
        
        for param in list(model.bert.encoder.layer[7].parameters()):
            param.requires_grad = True
            optimizer_grouped_parameters[0]["params"].append(param)
            
        for param in list(model.bert.encoder.layer[8].parameters()):
            param.requires_grad = True
            optimizer_grouped_parameters[0]["params"].append(param)
        
        for param in list(model.bert.encoder.layer[9].parameters()):
            param.requires_grad = True
            optimizer_grouped_parameters[0]["params"].append(param)  
            
        for param in list(model.bert.encoder.layer[10].parameters()):
            param.requires_grad = True
            optimizer_grouped_parameters[0]["params"].append(param)
            
        for param in list(model.bert.encoder.layer[11].parameters()):
            param.requires_grad = True
            optimizer_grouped_parameters[0]["params"].append(param)
            
        # for param in list(model.bert.encoder.layer[11].intermediate.parameters()):
        #     param.requires_grad = True
        #     optimizer_grouped_parameters[0]["params"].append(param)
        # for param in list(model.bert.encoder.layer[11].output.parameters()):
        #     param.requires_grad = True
        #     optimizer_grouped_parameters[0]["params"].append(param)
    return optimizer_grouped_parameters

In [28]:
# Set hyperparameters (optimizer, weight decay, learning rate)
FULL_FINETUNING = False
optimizer_grouped_parameters = get_hyperparameters(model, FULL_FINETUNING)
optimizer = torch.optim.AdamW(optimizer_grouped_parameters, lr=3e-5)
num_train_steps = int(len(train_dataset) / TRAIN_BATCH_SIZE * EPOCHS)
scheduler = get_linear_schedule_with_warmup(
    optimizer, 
    num_warmup_steps=0, 
    num_training_steps=num_train_steps
)

In [29]:
# Процесс обучения 
EPOCHS=15
for epoch in range(EPOCHS):
    model, train_loss, train_metrics = train_fn(train_data_loader, model, optimizer, _device, scheduler)
    val_loss, val_metrics = val_fn(val_data_loader, model, optimizer, _device, scheduler)
    print(f"Epoch: {epoch + 1}, Train_loss: {train_loss}, Val_loss: {val_loss}")
    print(f"Epoch: {epoch + 1}, Train_metrics: {train_metrics}")
    print(f"Epoch: {epoch + 1}, Val_metrics: {val_metrics}")

  0%|                                                                                           | 0/14 [00:00<?, ?it/s]C:\Users\andrey.NOUT\AppData\Roaming\Python\Python312\site-packages\seqeval\metrics\sequence_labeling.py:171: UserWarning: WP seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
C:\Users\andrey.NOUT\AppData\Roaming\Python\Python312\site-packages\seqeval\metrics\sequence_labeling.py:171: UserWarning: COUNT seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
C:\Users\andrey.NOUT\AppData\Roaming\Python\Python312\site-packages\seqeval\metrics\sequence_labeling.py:171: UserWarning: DIR seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
C:\Users\andrey.NOUT\AppData\Roaming\Python\Python312\site-packages\seqeval\metrics\sequence_labeling.py:171: UserWarning: SLD seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
C:\Users\andrey.NOUT\AppData\Roaming

Epoch: 1, Train_loss: 1.135519734450749, Val_loss: 0.6430875162283579
Epoch: 1, Train_metrics: {'precision': 0.16541494465936332, 'recall': 0.1216796566342314, 'accuracy_score': 0.7310467155612245}
Epoch: 1, Val_metrics: {'precision': 0.4930668167496066, 'recall': 0.3611059399793763, 'accuracy_score': 0.8321533203125}


100%|████████████████████████████████████████████████████████████████████████████████████| 6/6 [00:28<00:00,  4.79s/it]


Epoch: 2, Train_loss: 0.5250089530433927, Val_loss: 0.38424165546894073
Epoch: 2, Train_metrics: {'precision': 0.38199178024096775, 'recall': 0.3847829324477794, 'accuracy_score': 0.8507223323899872}
Epoch: 2, Val_metrics: {'precision': 0.41081421087641373, 'recall': 0.3564614138786435, 'accuracy_score': 0.8372802734375}


100%|████████████████████████████████████████████████████████████████████████████████████| 6/6 [00:30<00:00,  5.10s/it]


...

Epoch: 13, Train_loss: 0.07375201662736279, Val_loss: 0.10866193349162738
Epoch: 13, Train_metrics: {'precision': 0.6225431634742628, 'recall': 0.94242357945375, 'accuracy_score': 0.9131264200015944}
Epoch: 13, Val_metrics: {'precision': 0.6885269943681623, 'recall': 0.956557314092819, 'accuracy_score': 0.898193359375}


100%|████████████████████████████████████████████████████████████████████████████████████| 6/6 [00:28<00:00,  4.80s/it]


Epoch: 14, Train_loss: 0.07052471462105002, Val_loss: 0.11067127312223117
Epoch: 14, Train_metrics: {'precision': 0.6261272099017018, 'recall': 0.939028696512955, 'accuracy_score': 0.9141870615433673}
Epoch: 14, Val_metrics: {'precision': 0.686195413828067, 'recall': 0.9605542609089559, 'accuracy_score': 0.8980712890625}


100%|████████████████████████████████████████████████████████████████████████████████████| 6/6 [00:29<00:00,  4.84s/it]

Epoch: 15, Train_loss: 0.06890591659716197, Val_loss: 0.11055973917245865
Epoch: 15, Train_metrics: {'precision': 0.6249239604801263, 'recall': 0.9428508267145929, 'accuracy_score': 0.9142736318160077}
Epoch: 15, Val_metrics: {'precision': 0.6864593295870401, 'recall': 0.9605542609089559, 'accuracy_score': 0.8980305989583334}


In [30]:
# Достингнуте значения лоса. Что бы орентироваться  -учить еще или нет
# Epoch: 15, Train_loss: 0.0770851458821978, Val_loss: 0.10936438540617625

# Epoch: 15, Train_loss: 0.06890591659716197, Val_loss: 0.11055973917245865
# Epoch: 15, Train_metrics: {'precision': 0.6249239604801263, 'recall': 0.9428508267145929, 'accuracy_score': 0.9142736318160077}
# Epoch: 15, Val_metrics: {'precision': 0.6864593295870401, 'recall': 0.9605542609089559, 'accuracy_score': 0.8980305989583334}

In [38]:
model_path = str(pathlib.Path(MODEL_DIR) / FINETUNED_MODEL_NAME)

In [39]:
# Сохраняем модель после обучения

torch.save(model.state_dict(), model_path)

NameError: name 'model' is not defined

In [23]:
def prediction(test_sentence, model):
    # for i in list(string.punctuation):
    #     test_sentence = test_sentence.replace(i, ' ' + i)
    # test_sentence = test_sentence.split()
    #print(test_sentence)
    #from_name, to_name, value = self.label_interface.get_first_tag_occurence('Labels', 'Text')
    ds_raw2 = []
    # now tokenize chunks separately and add them to the dataset
    Token_inputs = _tokenizer.tokenize(test_sentence)
    item = {'id': 0, 
            'tokens': Token_inputs, 
            'tokens_ids': _tokenizer.encode(test_sentence, add_special_tokens=False), 
            'ner_tags': [0] * len(Token_inputs), 
            value: test_sentence}
    ds_raw2.append(item)

    ds_raw_ext2 = make_ds_raw_ext (ds_raw2)
    print(len(ds_raw2), len(ds_raw_ext2))
        
   
    test_dataset =  Dataset(ds_raw = ds_raw_ext2)
    #print(test_dataset[0]["target_tags"].size())
    tags = []
    scores = []
    with torch.no_grad():
        # Перебор датасета
        for index_data in range(test_dataset.__len__()):
            data = test_dataset[index_data]
            #print(data)
            for i, j in data.items():
                #print(i, j)
                data[i] = j.to(_device).unsqueeze(0)
            tag, _ = model(**data)
            print(tag.size())
            #print(tag)
            #print(le.inverse_transform(tag.argmax(2).cpu().numpy().reshape(-1))[1:len(Token_inputs)+1])
            tag2 = tag.argmax(2).cpu().numpy().reshape(-1)[1:len(Token_inputs)+1]
            score, max_indices  = torch.max(tag, dim=2)#.cpu().numpy().reshape(-1)[1:len(Token_inputs)+1]
            score = score.cpu().numpy().reshape(-1)[1:len(Token_inputs)+1]
            #print (score)
            score_norm = (score - np.min(score))/np.ptp(score)
            tags.append (tag2)
            scores.append(score_norm)
            #print (b)
            # print(tag.argmax(2).cpu().numpy().reshape(-1)[1:len(Token_inputs)+1])
            # for index in range(len(tag2)):
            #     print(Token_inputs[index], tag2[index])

    # Теперь нужно выровнять результат так как мы сделали это со сдвигом SHIFT_SIZE
    tags_result = []
    scores_result = []
    
    tokens_len = len (Token_inputs)
    print(tokens_len)
    chank_index = 0
    for token_index in range(0, tokens_len, MAX_LEN-SHIFT_SIZE):
        token_index_end = token_index+MAX_LEN
        tag = tags[chank_index]
        score = scores[chank_index]
        #print(len(tag))
        #print(token_index, max(tokens_len - token_index - len(tag),0))
        tag = np.pad(tag, (token_index, max(tokens_len - token_index - len(tag),0) ), 'constant', constant_values=(0, 0))
        tag = tag[0:tokens_len]
        tags[chank_index] = tag
        score = np.pad(score, (token_index, max(tokens_len - token_index - len(score),0) ), 'constant', constant_values=(0, 0))
        score = score[0:tokens_len]
        scores[chank_index] = score
        print(len(tag))
        chank_index = chank_index + 1

    print(len(tags[1]))
    tags_np = np.vstack( tags)
    scores_np = np.vstack( scores )
    scores_np_arg_max = np.argmax(scores_np, axis=0)
    tags_result = np.zeros(tokens_len)
    scores_result = np.zeros(tokens_len)
    for idxx, idxy in enumerate(scores_np_arg_max):
        tags_result[idxx] = tags_np[idxy, idxx]
        scores_result[idxx] = scores_np[idxy, idxx]
    print (tags_np.shape, scores_np_arg_max.shape)
    # print (tags_np.shape, scores_np.shape)
    # scores_np_max = np.max(scores_np, axis=0)
    # score_np_bool = (scores_np == scores_np_max)
    # with np.printoptions(threshold=np.inf):
    #     print(score_np_bool)
    #     print(scores_np[score_np_bool].shape)

    

    
    tag2 = VALUE_TOKEN +list(tags_result) + VALUE_TOKEN
    score_norm = [0] + list(scores_result) + [0]
    #encoding = tokenizer(text)
    return tag2, score_norm
    
            
            

In [24]:
# Получаем текст
task = tasks[56]
text = ner.preload_task_data(task, task['data'][value])
text = "Сводка Министерства обороны Российской Федерации о ходе проведения специальной военной операции на территории Украины 05.08.2022 г. Вооруженные Силы Российской Федерации продолжают специальную военную операцию на Украине. Высокоточными ракетами воздушного базирования ВКС России нанесены удары по пунктам временной дислокации двух гаубичных артиллерийских дивизионов 44-й артиллерийской бригады ВСУ и складам ракетно-артиллерийского вооружения в районе населенного пункта Новоивановка Запорожской области. Уничтожено до 150-ти военнослужащих, две американские гаубицы М777 и до полутора тысяч боеприпасов к ним, шесть гаубиц Мста-Б, восемь артиллерийских орудий различной модификации, около 350-ти снарядов для реактивных систем залпового огня Град, а также семь беспилотных летательных аппаратов.В Харьковской области девять местных жителей пострадали в результате конфликта с командирами подразделений 92-й механизированной бригады ВСУ.Командование данного соединения пыталось принудительно выселить мирных граждан из своих домов для размещения в них иностранных намников, а также оборудования на придомовых участках позиций военной техники и артиллерии.Для пресечения протестов националисты и иностранные намники открывали стрельбу, в результате которой четыре местных жителя в населенном пункте Шестаково и пять в Верхнем Салтове получили огнестрельные ранения различной степени тяжести. Продолжается нанесение ударов оперативно-тактической и армейской авиацией, ракетными войсками и артиллерией по военным объектам на территории Украины. За сутки поражено: четыре пункта управления, в том числе 58-й мотопехотной бригады в районе населенного пункта Артмовск, 54-й механизированной бригады в районе населенного пункта Северск Донецкой Народной Республики, пункт временной дислокации формирования националистов в районе населенного пункта Выводово Днепропетровской области, а также живая сила и военная техника ВСУ в 182-х районах. Уничтожено два склада ракетно-артиллерийского вооружения и боеприпасов в районе населенных пунктов Константиновка и Артмовск Донецкой Народной Республики. В ходе контрбатарейной борьбы в районах населенных пунктов Дзержинск, Новгородское и Николаевка Донецкой Народной Республики подавлены на огневых позициях три взвода гаубиц Д-30, а также в районе населенного пункта Чугуев Харьковской области взвод РСЗО Град. Российскими средствами противовоздушной обороны за сутки в воздухе сбито пять беспилотных летательных аппаратов в районах населенных пунктов Ясиноватая, Краснополье Донецкой Народной Республики, Букино, Сулиговка и Куриловка Харьковской области.Кроме того, перехвачены: три снаряда реактивных систем залпового огня в районах населенных пунктов Минеральное Донецкой Народной Республики и Изюм Харьковской области, а также в районе Алешковские Пески Херсонской области сбиты три баллистические ракеты Точка-У. Всего с начала проведения специальной военной операции уничтожено: 261 самолет, 145 вертолетов, 1685 беспилотных летательных аппаратов, 361 зенитный ракетный комплекс, 4254 танка и других боевых бронированных машин, 789 боевых машин реактивных систем залпового огня, 3267 орудий полевой артиллерии и минометов, а также 4716 единиц специальной военной автомобильной техники."
text = "Сводка Министерства обороны Российской Федерации о ходе проведения специальной военной операции на территории Украины 12.08.2022 г. Вооруженные Силы Российской Федерации продолжают специальную военную операцию на Украине. В результате наступательных действий союзных сил в районе Соледара потери 14-й механизированной бригады ВСУ составили более двух тысяч человек. Остатки личного состава бригады выведены украинским командованием в тыловые районы.   В районах населенных пунктов Опытное и Невельское Донецкой Народной Республики в результате высокоточных ударов ВКС России потери 56-й мотопехотной бригады ВСУ в живой силе превысили 70 процентов списочной численности.  Личный состав 23-го батальона бригады в полном составе самовольно оставил боевые позиции и убыл в тыловые районы.  Высокоточным ударом ВКС России по пункту временной дислокации 28-й механизированной бригады ВСУ в районе населенного пункта Новогригорьевка Николаевской области уничтожено до 100 боевиков и девять единиц военной техники.  В результате сосредоточенных огневых ударов по боевым позициям украинских националистов в районе Марьинки Донецкой народной республики, потери 3-го батальона 66-й механизированной бригады в живой силе составили более 50.    Кроме того, в районе населенного пункта Зайцево Донецкой Народной Республики в результате поражения опорного пункта 58-й мотопехотной бригады, уничтожено до 40 националистов и пять единиц автомобильной техники.  Продолжается нанесение ударов оперативно-тактической и армейской авиацией, ракетными войсками и артиллерией по военным объектам на территории Украины. За сутки поражены пять пунктов управления, в том числе 4-й танковой бригады в районе населенного пункта Залиман Харьковской области, 103-й бригады территориальной обороны в районе Николаевки Донецкой Народной Республики, а также живая сила, вооружение, военная и специальная техника в 157-ми районах. Уничтожены: склад горючего для украинской военной техники в районе Курахово Донецкой Народной Республики, шесть складов боеприпасов в районах Великое Артаково Николаевской области, Залиман Харьковской области, Краматорск, Зайцево, Красногоровка и Авдеевка Донецкой Народной Республики. В районе населенного пункта Часов Яр Донецкой Народной Республики ВКС России уничтожена поставленная США киевскому режиму радиолокационная станция контрбатарейной борьбы ANMPQ-64 производства США.  В рамках контрбатарейной борьбы подавлены четыре взвода реактивных систем залпового огня Град в районах населенных пунктов Серебрянка, Соледар Донецкой Народной Республики, Широкое и Великая Александровка Херсонской области. Кроме того, поражены два артиллерийских взвода орудий Акация и четыре взвода гаубиц Д-30 на огневых позициях в районах населенных пунктов Георгиевка, Константиновка, Новгородское, Веслое, Андреевка и Дзержинск Донецкой Народной Республики. Российскими средствами противовоздушной обороны за сутки уничтожено пять украинских беспилотных летательных аппаратов в районах населнных пунктов Егоровка, Спартак Донецкой Народной Республики, Архангеловка и Волхов Яр Харьковской области. Кроме того, сбиты в воздухе два реактивных снаряда системы залпового огня Хаймарс в районе Новой Каховки и четыре снаряда РСЗО Ольха в районе Чернобаевки Херсонской области. Всего с начала проведения специальной военной операции уничтожено: 267 самолетов, 146 вертолетов, 1732 беспилотных летательных аппарата, 365 зенитных ракетных комплексов, 4293 танка и других боевых бронированных машин, 797 боевых машин реактивных систем залпового огня, 3290 орудий полевой артиллерии и минометов, а также 4844 единицы специальной военной автомобильной техники."
text = "Сводка Министерства обороны Российской Федерации о ходе проведения специальной военной операции на территории Украины 06.09.2022 г.Часть 1  Вооруженные силы Украины в течение суток продолжали попытки атаковать на отдельных участках Николаево-Криворожского направления. ВКС России, ракетные войска и артиллерия наносят высокоточные удары по подразделениям и резервам ВСУ.  В районах населнных пунктов Висунск, Явкино, Березнеговатое и Червонополье поражены живая сила и военная техника в пунктах временной дислокации 61-й пехотной бригады, 35-й бригады морской пехоты и 17-й танковой бригады ВСУ. Также уничтожено три склада ракетно-артиллерийского вооружения и боеприпасов в районе Великое Артаково Николаевской области.  Высокоточными ракетами большой дальности морского базирования Калибр в районе населенного пункта Карповка Днепропетровской области уничтожено крупное хранилище топлива, предназначавшегося группировке украинских войск на Николаево-Криворожском направлении.  Истребительной авиацией ВКС России и средствами противовоздушной обороны сбиты три Су-25 воздушных сил Украины. Один Су-25 в районе населнного пункта Снигиревка и два украинских Су-25 в районе Мирное в Николаевской области.  Всего за сутки на Николаево-Криворожском направлении противник потерял 12 танков, 11 боевых машин пехоты и 8 других бронированных машин, 6 пикапов с крупнокалиберными пулеметами и более 210-ти военнослужащих.   Высокоточным оружием воздушного базирования поражены пункты временной дислокации подразделений 54-й и 93-й механизированных бригад ВСУ в районе города Артмовск Донецкой народной республики. Уничтожено до 250-ти военнослужащих и более 10 единиц автомобильной и бронированной техники.  В районе населенного пункта Веселянка Запорожской области высокоточными ударами ВКС России по пунктам временной дислокации 1-го батальона 65-й механизированной бригады ВСУ уничтожено до 100 боевиков и 15 единиц военной техники.  Из-за высоких потерь и прекращения ротации действующий в районе Зайцево Донецкой Народной Республики личный состав подразделений 53-й механизированной бригады ВСУ отказывается выполнять боевые задачи и группами покидает боевые позиции.   Продолжается нанесение ударов оперативно-тактической и армейской авиацией, ракетными войсками и артиллерией по военным объектам на территории Украины.  За сутки поражены три пункта управления ВСУ, в том числе 14-й механизированной бригады в районе Каменная Яруга Харьковской области, командно-наблюдательный пункт батальона 102-й бригады территориальной обороны в населенном пункте Полтавка Запорожской области, а также 52 артиллерийских подразделения, живая сила и военная техника ВСУ в 161-м районе.  В районе Григоровки Донецкой Народной Республики уничтожена пусковая установка украинского зенитного ракетного комплекса Бук-М1.   Средствами противовоздушной обороны за сутки сбиты 12 беспилотных летательных аппаратов в районах населнных пунктов Яковенково, Капитоловка, Липцы Харьковской области, Кирилловка, Сладкое, Валерьяновка, Новоандреевка Донецкой Народной Республики, Скадовск Херсонской области и города Херсон.  Также перехвачены баллистическая ракета Точка-У в районе населенного пункта Попасная Луганской Народной Республики и 11 снарядов реактивной системы залпового огня HIMARS производства США в районе Каховской ГЭС.   Всего с начала проведения специальной военной операции уничтожено: 290 самолетов, 152 вертолета, 1889 беспилотных летательных аппаратов, 373 зенитных ракетных комплекса, 4845 танков и других боевых бронированных машин, 825 боевых машин реактивных систем залпового огня, 3369 орудий полевой артиллерии и минометов, а также 5343 единицы специальной военной автомобильной техники.   "

In [25]:
text = """Вооруженные Силы Российской Федерации продолжают проведение специальной военной операции.💥 Подразделениями группировки войск «Север» нанесено поражение скоплениям живой силы и техники двух десантно-штурмовых, мотопехотной бригад ВСУ и двух бригад теробороны в районах населенных пунктов Юнаковка, Будки, Садки, Марьино Сумской области и Малые Проходы Харьковской области.

▪️ Потери ВСУ составили до 170 военнослужащих, две боевые бронированные машины, 16 автомобилей и пять орудий полевой артиллерии.
 
🔥 Подразделения группировки войск «Запад» улучшили тактическое положение. Нанесли поражение формированиям механизированной, егерской, штурмовой, аэромобильной бригад ВСУ и бригады теробороны в районах населенных пунктов Глущенково, Высшее Соленое, Дружелюбовка, Каменка, Ольговка, Соболевка, Купянск, Красное Первое и Московка Харьковской области.

▪️ Противник потерял свыше 220 военнослужащих, бронетранспортер М113 производства США, реактивную пусковую установку «Heron» производства Хорватии, четыре автомобиля и пять артиллерийских орудий, из них два западного производства. Уничтожены три склада боеприпасов.
 
📍 Подразделения «Южной» группировки войск заняли более выгодные рубежи и позиции. Нанесли поражение живой силе и технике двух механизированных, мотопехотной, горно-штурмовой, аэромобильной бригад ВСУ, бригады спецназначения и бригады теробороны в районах населенных пунктов Северск, Серебрянка, Звановка, Клиновое, Веролюбовка, Александро-Калиново и Константиновка Донецкой Народной Республики.

▪️ Потери украинских вооруженных формирований составили свыше 235 боевиков, бронетранспортер М113 производства США, три автомобиля и четыре орудия полевой артиллерии, в том числе два западного производства. Уничтожены четыре склада боеприпасов и станция радиоэлектронной борьбы.
 Подразделения группировки войск «Центр» улучшили положение по переднему краю. Нанесли поражение формированиям двух механизированных, воздушно-десантной бригад ВСУ, бригады спецназначения и бригады нацгвардии в районах населенных пунктов Полтавка, Новоэкономическое, Димитров, Новопавловка, Тарасовка, Елизаветовка, Лысовка, Березовка, Александрополь, Миролюбовка и Зверево Донецкой Народной Республики.

▪️ Противник потерял до 460 военнослужащих, танк AMX производства Франции, четыре боевые бронированные машины, три автомобиля и артиллерийское орудие.
 
↗️ Подразделения группировки войск «Восток» продолжили продвижение в глубину обороны противника. Нанесли поражение живой силе и технике механизированной, егерской, аэромобильной бригад ВСУ и бригады теробороны в районах населенных пунктов Зеленый Гай, Карла Маркса, Отрадное, Богатырь Донецкой Народной Республики и Гуляйполе Запорожской области.

▪️ Потери противника составили до 170 военнослужащих, боевая бронированная машина, восемь автомобилей, четыре орудия полевой артиллерии, в том числе два производства стран НАТО.
 
💥 Подразделения группировки войск «Днепр» нанесли поражение формированиям механизированной бригады, двух бригад береговой обороны ВСУ и бригады теробороны в районах Павловка, Каменское, Новоандреевка, Малая Токмачка Запорожской области, Ромашково Никольское, Велетенское, Садовое и Антоновка Херсонской области.

▪️ Противник потерял до 60 военнослужащих, бронеавтомобиль HMMWV производства США и шесть автомобилей. Уничтожены станция радиоэлектронной борьбы и склад боеприпасов.
 
✈️ Оперативно-тактической авиацией, ударными беспилотными летательными аппаратами, ракетными войсками и артиллерией группировок войск Вооруженных Сил Российской Федерации поражены объекты инфраструктуры военных аэродромов, склады боеприпасов, пункты управления безэкипажных катеров и беспилотных летательных аппаратов, завод по производству боеприпасов и взрывчатых веществ, а также пункты временной дислокации украинских вооруженных формирований и иностранных наемников в 152 районах.

🎯 Средствами противовоздушной обороны уничтожены семь управляемых авиационных бомб JDAM и реактивный снаряд системы залпового огня HIMARS производства США, а также 292 беспилотных летательных аппарата самолетного типа."""

In [53]:
text="""Министерства обороны Российской Федерации о ходе проведения специальной военной операции по состоянию на 5 мая 2025 г.
 
👮‍♂️ Вооруженные Силы Российской Федерации продолжают проведение специальной военной операции.
 
💥 Подразделениями группировки войск «Север» нанесено поражение скоплениям живой силы и техники механизированной, танковой, егерской бригад ВСУ и двух бригад теробороны в районах населенных пунктов Садки, Рясное, Великая Писаревка Сумской области и Гранов Харьковской области.

▪️ Потери ВСУ составили до 150 военнослужащих, три танка, две боевые бронированные машины и шесть автомобилей. Уничтожен склад боеприпасов.
 
📍 Подразделения группировки войск «Запад» заняли более выгодные рубежи и позиции. Нанесли поражение формированиям двух механизированных, горно-штурмовой, штурмовой бригад ВСУ и бригады теробороны в районах населенных пунктов Пески, Купянск, Григоровка, Кутьковка Харьковской области и Карповка Донецкой Народной Республики.

▪️ Противник потерял свыше 225 военнослужащих, боевую бронированную машину, шесть автомобилей и два артиллерийских орудия западного производства. Уничтожены три склада боеприпасов.
 
📍 Подразделения «Южной» группировки войск улучшили тактическое положение. Нанесли поражение живой силе и технике двух механизированных, аэромобильной бригад ВСУ и бригады теробороны в районах населенных пунктов Серебрянка, Дружковка, Северск и Заря Донецкой Народной Республики.

▪️ Потери украинских вооруженных формирований составили свыше 315 военнослужащих, две боевые бронированные машины и восемь орудий полевой артиллерии.
 
📍 Подразделения группировки войск «Центр» улучшили положение по переднему краю. Нанесли поражение формированиям двух механизированных, егерской бригад ВСУ, бригады спецназначения и двух бригад нацгвардии в районах населенных пунктов Удачное, Димитров, Новопавловка, Новосергеевка и Гродовка Донецкой Народной Республики.

▪️ Противник потерял до 465 военнослужащих, танк, четыре боевые бронированные машины, шесть автомобилей и четыре артиллерийских орудия.
Подразделения группировки войск «Восток» продолжили продвижение в глубину обороны противника. Нанесли поражение живой силе и технике двух механизированных бригад ВСУ, бригады морской пехоты и бригады теробороны в районах населенных пунктов Богатырь, Федоровка, Комар и Новополь Донецкой Народной Республики.

▪️ Потери противника составили до 170 военнослужащих, боевая бронированная машина, пять автомобилей и четыре орудия полевой артиллерии. Уничтожены две станции радиоэлектронной борьбы.
 
💥 Подразделения группировки войск «Днепр» нанесли поражение формированиям механизированной бригады и двух бригад береговой обороны ВСУ в районах населенных пунктов Малая Токмачка, Павловка Запорожской области и Антоновка Херсонской области.

▪️ Противник потерял до 70 военнослужащих, пять автомобилей, пусковую установку реактивной системы залпового огня HIMARS производства США и два орудия полевой артиллерии. Уничтожены две станции радиоэлектронной борьбы и склад боеприпасов.
 
✈️ Оперативно-тактической авиацией, ударными беспилотными летательными аппаратами, ракетными войсками и артиллерией группировок войск Вооруженных Сил Российской Федерации поражены объекты инфраструктуры военного аэродрома, ремонтного предприятия авиационной промышленности, склады боеприпасов, а также пункты временной дислокации украинских вооруженных формирований и иностранных наемников в 142 районах.
 
🎯 Средствами противовоздушной обороны уничтожены четыре управляемые авиационные бомбы JDAM и четыре реактивных снаряда системы залпового огня HIMARS производства США, а также 123 беспилотных летательных аппарата самолетного типа.
 
📊 Всего с начала проведения специальной военной операции уничтожены:

▫️ 662 самолета, 
▫️ 283 вертолета, 
▫️ 55 452 беспилотных летательных аппарата, 
▫️ 605 зенитных ракетных комплексов, 
▫️ 23 153 танка и других боевых бронированных машин, 
▫️ 1 558 боевых машин реактивных систем залпового огня, 
▫️ 24 435 орудий полевой артиллерии и минометов, 
▫️ 35 005 единиц специальной военной автомобильной техники.
"""

In [54]:
 text = re.sub(r"[^а-яА-Яa-zA-Z0-9 \-\,\:\.]", "", text)

In [55]:
text

'Министерства обороны Российской Федерации о ходе проведения специальной военной операции по состоянию на 5 мая 2025 г.  Вооруженные Силы Российской Федерации продолжают проведение специальной военной операции.  Подразделениями группировки войск Север нанесено поражение скоплениям живой силы и техники механизированной, танковой, егерской бригад ВСУ и двух бригад теробороны в районах населенных пунктов Садки, Рясное, Великая Писаревка Сумской области и Гранов Харьковской области. Потери ВСУ составили до 150 военнослужащих, три танка, две боевые бронированные машины и шесть автомобилей. Уничтожен склад боеприпасов.  Подразделения группировки войск Запад заняли более выгодные рубежи и позиции. Нанесли поражение формированиям двух механизированных, горно-штурмовой, штурмовой бригад ВСУ и бригады теробороны в районах населенных пунктов Пески, Купянск, Григоровка, Кутьковка Харьковской области и Карповка Донецкой Народной Республики. Противник потерял свыше 225 военнослужащих, боевую брониро

In [56]:
model = _model.to(_device)
tags, score_norm = prediction(text, model)

1 4
torch.Size([1, 512, 10])
torch.Size([1, 512, 10])
torch.Size([1, 512, 10])
torch.Size([1, 512, 10])
1047
1047
1047
1047
1047
1047
(4, 1047) (1047,)


In [57]:
def generate_id(length=10):
    characters = string.ascii_letters + string.digits
    return ''.join(random.choice(characters) for i in range(length))

In [62]:
# Создаем представление данных, для отправки в LabelStudio на запрос predict
li = ner.label_interface
from_name, to_name, value = li.get_first_tag_occurence('Labels', 'Text')
encoding = _tokenizer(text)
tokens = encoding.tokens()
word_index_prev = -1
prev_label = 'O'
predictions = []
results = []
avg_score = 0
for index in range(len(tags)):
    word_index = encoding.token_to_word(index)
    # не для всех токенов есть слова
    if word_index != None:
        #print (word_index)
        # если слово новое (не повтор - одно слово может состоять из нескольких токенов)
        if word_index_prev != word_index:
            label = id_to_label[tags[index]]
            score = score_norm[index]
            if label != 'O' and score >=0.0:
                # начало и конец слова
                start, end = encoding.word_to_chars(word_index)
                if prev_label != label:
                    # Если это новая метка - то добавляем
                    results.append({
                            'id': str(uuid4()),#generate_id(),
                            'from_name': from_name,
                            'to_name': to_name,
                            'type': 'labels',
                            'value': {
                                'start': start,
                                'end': end,
                                'labels': [label],
                                'text': text[start:end]
                            },
                            'score': score
                        })
                else:
                    # Если метка уже была - то это многословная метка, значит просто продлеваем предыдущую
                    results[-1]["value"]["end"] = end
                    results[-1]["value"]["text"] =  text[results[-1]["value"]["start"]:end]
                avg_score += score
            prev_label = label
        word_index_prev = word_index
if results:
    predictions.append({
        'result': results,
        'score': avg_score / len(results),
        'model_version': ner.get('model_version')
    })
# Добавляем связи между тегами ['UNIT', 'WP', 'SLD', 'CAPT'] и тегом с количеством COUNT'
# Исходим из того, что следующий после тега COUNT тег связан с этим COUNT

for index, item in enumerate ( predictions[0]['result']):
    #print(item)
    label = item['value']['labels'][0]
    if label == 'COUNT':
        # Берем следующий итем
        if index < len(predictions[0]['result']) - 1:
            next_item = predictions[0]['result'][index+1]
            next_label = next_item['value']['labels'][0]
            if next_label in ['UNIT', 'WP', 'SLD', 'CAPT']:
                # Добавляем связь между следющим итемом и итемом с числом
                rel = { 'from_id': next_item['id'],
                        'to_id': item['id'],
                        'type': 'relation',
                        'direction': 'right',
                        'labels': ["has_count"],
                        'region': item
                    }
                relations = []
                relations.append(rel)
                next_item['relations'] = relations
# if relations:
#      predictions[0]['result'].append(relations)
    
        

In [63]:
predictions

[{'result': [{'id': '8a4ff3e2-0ebf-41a9-88f0-5fa379088b9c',
    'from_name': 'label',
    'to_name': 'text',
    'type': 'labels',
    'value': {'start': 245, 'end': 250, 'labels': ['DIR'], 'text': 'Север'},
    'score': 0.13899622857570648},
   {'id': '8e671bf1-c8c2-4ce9-a9e7-b674c55c49c2',
    'from_name': 'label',
    'to_name': 'text',
    'type': 'labels',
    'value': {'start': 404, 'end': 409, 'labels': ['LOC'], 'text': 'Садки'},
    'score': 0.6412344574928284},
   {'id': '8059a99b-3095-4b22-8409-47429af4a546',
    'from_name': 'label',
    'to_name': 'text',
    'type': 'labels',
    'value': {'start': 411, 'end': 417, 'labels': ['LOC'], 'text': 'Рясное'},
    'score': 0.6719184517860413},
   {'id': '46430a57-5299-46f5-9c75-dc145c5b8b93',
    'from_name': 'label',
    'to_name': 'text',
    'type': 'labels',
    'value': {'start': 419,
     'end': 436,
     'labels': ['LOC'],
     'text': 'Великая Писаревка'},
    'score': 0.6683118939399719},
   {'id': '0367d5ac-be37-4d79-af0

In [64]:
prev_label = ''
prev_item = None
next_label = ''
for index, item in enumerate ( predictions[0]['result']):
    #print(item)
    label = item['value']['labels'][0]
    # Берем следующий итем
    if index < len(predictions[0]['result']) - 1:
        next_item = predictions[0]['result'][index+1]
        next_label = next_item['value']['labels'][0]
    if label == 'DIR':
        sys.stdout.write (f"\r\nНаправление:  {item['value']['text']}")
    if label == 'LOC':
        if label != prev_label:
            sys.stdout.write (f"\r\nНаселенные пункты: ")
        sys.stdout.write (f"{item['value']['text']}")
        if label == next_label:
            sys.stdout.write (f", ")
            
    if label in ['UNIT', 'WP', 'SLD', 'CAPT']:
        if label != prev_label:
            sys.stdout.write (f"\r\n")
        sys.stdout.write (f"  {item['value']['text']}")
        if label == next_label:
            sys.stdout.write (f", ")
        if "relations" in item:
            sys.stdout.write (f" - {item['relations'][0]['region']['value']['text']}")
    



    prev_label = label
    prev_item = item
        


Направление:  Север
Населенные пункты: Садки, Рясное, Великая Писаревка, Гранов
  военнослужащих - 150
  танка - три
  боевые бронированные машины - две
  автомобилей - шесть
  склад боеприпасов
Направление:  Запад
Населенные пункты: Пески, Купянск, Григоровка, Кутьковка, Карповка
  военнослужащих - 225
  боевую бронированную машину
  автомобилей - шесть
  артиллерийских орудия западного производства - два
  склада боеприпасов - три
Направление:  Южной
Населенные пункты: Серебрянка, Дружковка, Северск, Заря
  военнослужащих - 315
  боевые бронированные машины - две
  орудий полевой артиллерии - восемь
Направление:  Центр
Населенные пункты: Удачное, Димитров, Новопавловка, Новосергеевка, Гродовка
  военнослужащих - 465
  боевые бронированные машины - четыре
  автомобилей - шесть
  артиллерийских орудия - четыре
Населенные пункты: Богатырь, Федоровка, Комар, Новополь
  военнослужащих - 170
  боевая бронированная машина
  автомобилей - пять
  орудия полевой артиллерии - четыре
  станции 

In [108]:
reload_model()

Some weights of BertModel were not initialized from the model checkpoint at Babelscape/wikineural-multilingual-ner and are newly initialized: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [40]:
model_cpu = NERBertModel(num_tag=len(label_to_id))
model_cpu.load_state_dict(torch.load(model_path, weights_only=True, map_location=torch.device('cpu')))

Some weights of BertModel were not initialized from the model checkpoint at Babelscape/wikineural-multilingual-ner and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


<All keys matched successfully>

In [239]:
model_cpu

NERBertModel(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(119547, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_aff

In [241]:
transformers.__version__

'4.39.1'